In [3]:
import tensorflow as tf
import os

base_dir = "C:/Users/Administrator/Downloads/FruitsAndVegDetect-main/Fruit-Images-Dataset-master/Training"
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=30,
    width_shift_range= 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    validation_split = 0.1,
    fill_mode = 'nearest',
)

train_generator = datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE,IMAGE_SIZE),batch_size=BATCH_SIZE,subset='training')


labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt','w') as f:
    f.write(labels)

Found 60955 images belonging to 131 classes.


In [ ]:


IMG_SHAPE = (IMAGE_SIZE,IMAGE_SIZE,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top = False,weights = 'imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131,activation='softmax')
])

model.compile(optimizer = tf.keras.optimizers.Adam(),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
epochs = 10
model.fit(train_generator,epochs=epochs,steps_per_epoch=200)



In [ ]:

# Define the path to your SavedModel directory

# Define the path where you want to save the .tflite file

save_model_dir = "E:\\Deep Learning Projects\\Model\\vegetables"
saved_model = tf.saved_model.load(model,save_model_dir)
tflite_model_path = "E:\\Deep Learning Projects\\Model\\vegModel"

# Load the SavedModel


# Convert the SavedModel to a TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model)
tflite_model = converter.convert()

# Save the TFLite model to the specified path
with open('vegModels.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"Model saved as {tflite_model_path}")
